<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso3/ciclo3/3_optimizacion_hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=14reVO1X6LsjqJ3cFgoeHxxddZVGfZn3t" width="100%">

# Optimización de Hiperparámetros
---

En este notebook veremos la necesidad de la optimización de hiperparámetros y algunas herramientas populares en _Python_.

Comenzamos instalando e importando las librerías necesarias:

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

## **1. Motivación**
---

La selección de hiperparámetros es crucial en el modelamiento de aprendizaje automático por varias razones:

- **Mejora la calidad del modelo**: La correcta selección de hiperparámetros puede mejorar significativamente la precisión, el rendimiento y la robustez del modelo.
- **Evita el sobreajuste**: El sobreajuste se produce cuando un modelo se ajusta demasiado a los datos de entrenamiento, dando como resultado un rendimiento pobre en datos desconocidos. La selección adecuada de hiperparámetros puede ayudar a evitar el sobreajuste y mejorar la generalización del modelo.
- **Mejora la eficiencia**: La búsqueda de hiperparámetros puede ser costosa en términos de tiempo de computación y recursos. Sin embargo, es importante hacerla para obtener el mejor modelo posible.

Es importante recordar las diferencias entre parámetros e hiperparámetros de un modelo:

| Parámetros | Hiperparámetros |
| --- | --- |
| Requeridos para hacer predicciones | Requeridos para estimar los parámetros |
| Se estiman con algoritmos de optimización | Se estiman con algoritmos de búsqueda |
| Se encuentran en el entrenamiento | Deben ser ajustados manualmente |
| Los parámetros encontrados determinan las predicciones | Los hiperparámetros determinan el entrenamiento |

En _Python_ existen distintas librerías para optimización de hiperparámetros. Vamos a ver un ejemplo sobre un conjunto de datos sintético para regresión:

In [ ]:
x = np.random.uniform(
    low=-1,
    high=1,
    size=(2000, 1),
    )
y = np.cos(5 * x) * x ** 2 + np.random.normal(
    loc=0,
    scale=0.05,
    size=(2000, 1),
    )

Visualizamos el conjunto de datos:

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x, y, alpha=0.1)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
fig.show()

Dividimos el conjunto de datos en entrenamiento y prueba para evaluar la generalización del modelo:

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

En este caso vamos a entrenar un modelo de máquina de soporte vectorial para regresión y a evaluar el desempeño por medio del $r^2$:

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import r2_score

Veamos el desempeño del modelo de SVR con un kernel lineal:

In [ ]:
model = SVR(kernel="linear").fit(x_train, y_train.ravel())

Veamos el desempeño del modelo:

In [ ]:
y_pred = model.predict(x_test)
print(r2_score(y_test, y_pred))

Veamos este resultado de forma gráfica:

In [ ]:
fig, ax = plt.subplots()
x_range = np.linspace(-1, 1, 100).reshape(-1, 1)
y_pred = model.predict(x_range)
ax.scatter(x, y, alpha=0.1, label="data")
ax.plot(x_range, y_pred, label="predictions")
ax.legend()
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
fig.show()

Como podemos ver, obtenemos un modelo que no se ajusta muy bien a los datos. Veamos cómo optimizar los hiperparámetros con distintas estrategias y librerías:

## **2. Grid Search**
---

Grid search es un enfoque de búsqueda exhaustiva de hiperparámetros en el aprendizaje automático. El objetivo de Grid Search es encontrar la combinación óptima de hiperparámetros para un modelo dado. En esta búsqueda se especifican los valores posibles para cada hiperparámetro y luego se prueban todas las combinaciones posibles de esos valores. Por ejemplo, si se tienen dos hiperparámetros, cada uno con tres posibles valores, entonces Grid Search probará 9 combinaciones diferentes en total.

Para cada combinación de hiperparámetros se entrena un modelo con esos hiperparámetros y se evalúa su rendimiento en un conjunto de datos de prueba. Finalmente, se selecciona la combinación de hiperparámetros que produce el mejor rendimiento en el conjunto de prueba.

Grid search es una forma sencilla y efectiva de seleccionar hiperparámetros, pero puede ser costosa en términos de tiempo de computación y recursos, especialmente cuando se tienen muchos hiperparámetros y muchos posibles valores para cada uno de ellos. Por lo tanto, a veces es recomendable usar métodos más sofisticados de búsqueda de hiperparámetros, como la búsqueda aleatoria o el optimizador bayesiano.

Veamos cómo usar Grid Search en `sklearn`:

In [ ]:
from sklearn.model_selection import GridSearchCV

También importamos la siguiente función para seleccionar la métrica de desempeño:

In [ ]:
from sklearn.metrics import make_scorer

Definimos las combinaciones de hiperparámetros que vamos a explorar:

In [ ]:
param_grid = {
    "kernel": ["rbf", "poly", "linear"],
    "C": [1.0, 0.1, 0.01, 0.01],
    "gamma": [1.0, 0.1, 0.01, 0.01]
}

Realizamos la exploración:

In [ ]:
gsearch = GridSearchCV(
        estimator=SVR(),
        param_grid=param_grid,
        scoring=make_scorer(r2_score, greater_is_better=True)
        ).fit(x_train, y_train.ravel())

Veamos los resultados:

In [ ]:
display(pd.DataFrame(gsearch.cv_results_))

Obtenemos el mejor modelo:

In [ ]:
model = gsearch.best_estimator_

Evaluamos su desempeño en el conjunto de datos de prueba:

In [ ]:
y_pred = model.predict(x_test)
print(r2_score(y_test, y_pred))

Veamos este resultado de forma gráfica:

In [ ]:
fig, ax = plt.subplots()
x_range = np.linspace(-1, 1, 100).reshape(-1, 1)
y_pred = model.predict(x_range)
ax.scatter(x, y, alpha=0.1, label="data")
ax.plot(x_range, y_pred, label="predictions")
ax.legend()
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
fig.show()

Como podemos ver, el resultado presenta un mejor ajuste sobre los datos.

## **3. Random Search**
---

Random Search es un enfoque para seleccionar hiperparámetros en el aprendizaje automático. En lugar de probar todas las combinaciones posibles de hiperparámetros, como en Grid Search, Random Search selecciona aleatoriamente combinaciones de hiperparámetros para entrenar y evaluar modelos.

<img src="https://drive.google.com/uc?export=view&id=1m8uUfuYDG76uLHPTRryQP3w9-9stKa5z" width="80%">

En Random Search se especifican los valores posibles para cada hiperparámetro y luego se generan combinaciones aleatorias de esos valores. Por ejemplo, si se tienen dos hiperparámetros, cada uno con tres posibles valores, entonces Random Search generará combinaciones aleatorias de esos valores y entrenará y evaluará un modelo con cada combinación.

Después de un número determinado de iteraciones, Random Search seleccionará la combinación de hiperparámetros que produjo el mejor rendimiento en el conjunto de prueba.

Random Search es un enfoque más eficiente que Grid Search en términos de tiempo de computación y recursos, especialmente cuando se tienen muchos hiperparámetros y muchos posibles valores para cada uno de ellos. Además, Random Search a menudo es más efectivo que Grid Search en encontrar la combinación óptima de hiperparámetros. Sin embargo, la eficacia de Random Search depende del número de iteraciones y la distribución de valores posibles para cada hiperparámetro.

Veamos cómo podemos usarlo:

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

Su uso es muy parecido al de Grid Search, no obstante, podemos definir distribuciones sobre hiperparámetros en lugar de valores fijos, por ejemplo usando una distribución:

In [ ]:
from scipy.stats import halfnorm

Definimos las distribuciones de hiperparámetros:

In [ ]:
param_grid = {
    "kernel": ["rbf", "poly", "linear"],
    "C": halfnorm(loc=0, scale=0.5),
    "gamma": halfnorm(loc=1, scale=0.5)
}

Entrenamos el modelo:

In [ ]:
rsearch = RandomizedSearchCV(
        estimator=SVR(),
        param_distributions=param_grid,
        n_iter=30,
        scoring=make_scorer(r2_score, greater_is_better=True)
        ).fit(x_train, y_train.ravel())

Veamos los resultados:

In [ ]:
display(pd.DataFrame(rsearch.cv_results_))

Obtenemos el mejor modelo:

In [ ]:
model = rsearch.best_estimator_

Evaluamos su desempeño en el conjunto de datos de prueba:

In [ ]:
y_pred = model.predict(x_test)
print(r2_score(y_test, y_pred))

Veamos este resultado de forma gráfica:

In [ ]:
fig, ax = plt.subplots()
x_range = np.linspace(-1, 1, 100).reshape(-1, 1)
y_pred = model.predict(x_range)
ax.scatter(x, y, alpha=0.1, label="data")
ax.plot(x_range, y_pred, label="predictions")
ax.legend()
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
fig.show()

## **4. Optuna**
---

`optuna` es una biblioteca de software de código abierto para la optimización de hiperparámetros en el aprendizaje automático. Optuna ofrece una interfaz fácil de usar para realizar búsquedas de hiperparámetros de manera efectiva y eficiente.

`optuna` utiliza un enfoque de optimización bayesiana para seleccionar hiperparámetros. En lugar de probar todas las combinaciones posibles de hiperparámetros o seleccionarlas al azar, como en Grid Search o Random Search respectivamente, `optuna` utiliza una distribución probabilística para representar la incertidumbre sobre la optimización de hiperparámetros.

<img src="https://drive.google.com/uc?export=view&id=1lW-f98Od3zYxbfC1VQxarzcYxl6Lf-CF" width="80%">

A medida que se entrenan y evalúan modelos con diferentes combinaciones de hiperparámetros, Optuna actualiza su distribución probabilística y se concentra en las áreas más prometedoras del espacio de búsqueda de hiperparámetros. Esto permite a `optuna` explorar de manera eficiente el espacio de búsqueda y encontrar la combinación óptima de hiperparámetros con menos iteraciones que Grid Search o Random Search.

Además de la optimización bayesiana, `optuna` también ofrece otras funciones útiles, como la gestión de experimentos, la integración con distintos marcos de aprendizaje automático y la visualización de resultados. Optuna también puede ser usado de forma distribuida con distintos `workers` (nodos de procesamiento) y con un almacenamiento compartido, lo que permite su uso en grandes cantidades de datos:

<img src="https://drive.google.com/uc?export=view&id=13sF66PhaS5VwsJxC4NnpUcbs6gWjGjW0" width="80%">

Veamos cómo instalar `optuna`:

In [ ]:
!pip install optuna

El uso de `optuna` es sencillo, debemos definir una función objetivo a optimizar, en este caso será el $r^2$ sobre el conjunto de test. La función debe recibir como parámetro `trial`, el cual es un objeto de `optuna` que nos permitirá extraer hiperparámetros de forma controlada y con distintos tipos de distribuciones.

En este ejemplo, `suggest_float` nos permite extraer números reales en un rango dado, mientras que el parámetro `log` permite controlar si se realiza en escala logarítmica:

In [ ]:
def objective(trial):
    gamma = trial.suggest_float("gamma", 0.01, 10, log=True)
    c = trial.suggest_float("C", 0.01, 10, log=True)
    kernel = trial.suggest_categorical("kernel", ["rbf", "poly", "linear"])
    model = SVR(C=c, kernel=kernel, gamma=gamma).fit(x_train, y_train.ravel())
    y_pred = model.predict(x_test)
    score = r2_score(y_test, y_pred)
    return score

Ahora importamos `optuna`:

In [ ]:
import optuna

Para usar optuna debemos crear un estudio, para ello especificamos:

- `direction`: se específica si maximizamos o minimizamos.
- `storage`: tipo de almacenamiento para los resultados.
- `study_name`: nombre del estudio.

In [ ]:
study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///hp.db",
    study_name="svm",
    )

Ejecutamos la exploración, para ello especificamos lo siguiente:

- `func`: función a optimizar.
- `n_trials`: número de modelos a entrenar.
- `n_jobs`: número de nodos de procesamiento (-1 indica usar el máximo posible).

In [ ]:
study.optimize(func=objective, n_trials=100, n_jobs=-1)

Extraemos los mejores parámetros y el $r^2$ obtenido:

In [ ]:
params = study.best_params
print(params)

In [ ]:
score = study.best_value
print(score)

Podemos entrenar un modelo con estos parámetros:

In [ ]:
model = SVR(**params).fit(x_train, y_train.ravel())

Evaluamos su desempeño en el conjunto de datos de prueba:

In [ ]:
y_pred = model.predict(x_test)
print(r2_score(y_test, y_pred))

Veamos este resultado de forma gráfica:

In [ ]:
fig, ax = plt.subplots()
x_range = np.linspace(-1, 1, 100).reshape(-1, 1)
y_pred = model.predict(x_range)
ax.scatter(x, y, alpha=0.1, label="data")
ax.plot(x_range, y_pred, label="predictions")
ax.legend()
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
fig.show()

Adicionalmente, `optuna` dispone de un tablero para visualizar la exploración de hiperparámetros. Veamos cómo instalarlo:

In [ ]:
!pip install optuna-dashboard

Su uso es sencillo. Básicamente debemos lanzar el dashboard con la base de datos que fue creada durante la exploración

In [ ]:
command = """
optuna-dashboard \
        --port 5000 \
        sqlite:///hp.db &
"""
get_ipython().system_raw(command)

Al igual que con `mlflow`, debemos usar `ngrok` para poder acceder al tablero:

In [ ]:
!pip install pyngrok

Ahora debe agregar su token de `ngrok`:

In [ ]:
token = "" # Agregue el token dentro de las comillas
os.environ["NGROK_TOKEN"] = token

Nos autenticamos en ngrok:

In [ ]:
!ngrok authtoken $NGROK_TOKEN

Ahora, lanzamos la conexión con ngrok:

In [ ]:
from pyngrok import ngrok
ngrok.connect(5000, "http")

Este tablero le dará acceso a distintas visualizaciones que permitirán observar cómo es la optimización, importancia y dependencias entre hiperparámetros.

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [Hyperparameter Tuning in Python: a Complete Guide](https://neptune.ai/blog/hyperparameter-tuning-in-python-complete-guide)
- [Grid Search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
- [Random Search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
- [Optuna](https://optuna.org/)

## Créditos
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Asistente docente**:

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Diseño de imágenes:**
- [Brian Chaparro Cetina](mailto:bchaparro@unal.edu.co).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*